In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import time
import os
import copy
import cv2
import pydicom
from itertools import product
import pandas as pd

In [ ]:
basePath = "/kaggle/input"
rootFolder = f"{basePath}/rsna-miccai-brain-tumor-radiogenomic-classification"
trainFolder = rootFolder + "/train"
testFolder = rootFolder + "/test"
trainLabels = rootFolder + "/train_labels.csv"
types=["FLAIR", "T1w", "T1wCE", "T2w"]
model_base_path = f"{basePath}/2dvgg19pretrained/miccai-0-1065"
outputFolder = f"/kaggle/working/"

In [ ]:
train_df = pd.read_csv(trainLabels, dtype=str)
train_df.head(5)

In [ ]:
train_df.to_csv('/kaggle/working/test.csv', index = False)

In [ ]:
def getChildNames(path):
        return os.listdir(path)

In [ ]:
def getPathDataset(dsType, ):
    
    
    pathDS = []
    totalcases = 0
    BratsIds = []
    labels = []
    if dsType == "train":
        totalCases = train_df.shape[0]
        BratsIds = train_df.BraTS21ID.values
        labels = train_df.MGMT_value.values
    elif dsType == "test":
        BratsIds = [name for name in getChildNames(f"{rootFolder}/{dsType}")]
        totalCases = len(BratsIds)
        labels = [-1] * totalCases

    for index in range(totalCases):
        caseId = BratsIds[index]
        label = labels[index]
        for _type in types:
        #pathDS[caseId][_type] = []
            tmpFolderPath = f"{rootFolder}/{dsType}/{caseId}/{_type}/"
            allFiles = getChildNames(tmpFolderPath)
            allFiles = sorted(
                  allFiles, 
                  key=lambda x: int(x[:-4].split("-")[-1]),
              )
            allFiles = [(tmpFolderPath + name, caseId, label, _type) for name in allFiles]
            pathDS += allFiles

    pathDS = pd.DataFrame(pathDS, columns = ["FilePath","CaseId","MGMT_Value","Modality"])
    return pathDS

In [ ]:
def load_dicom(path):
    dicom = pydicom.read_file(path)
    data = dicom.pixel_array
    
    data = data - np.min(data)
    if np.max(data) != 0:
        data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data

In [ ]:
trainPathDS = getPathDataset("train")

In [ ]:
testPathDS = getPathDataset("test")

In [ ]:
testPathDS.head()

In [ ]:
trainPathDS.to_csv(f"{outputFolder}/trainPathDS.csv")

### 3D slice Count

In [ ]:
%%time
slice_info_3d = []

totalCases = train_df.shape[0]
BratsIds = train_df.BraTS21ID.values
labels = train_df.MGMT_value.values


for index in range(totalCases):
    caseId = BratsIds[index]
    label = labels[index]
    for _type in types:
    #pathDS[caseId][_type] = []
        tmpFolderPath = f"{rootFolder}/train/{caseId}/{_type}/"
        allFiles = getChildNames(tmpFolderPath)
        
        totalSlices = len(allFiles)
        H,W = load_dicom(f"{tmpFolderPath}/{allFiles[0]}").shape
        
        slice_info_3d.append((caseId, label, _type, f"{H}x{W}", totalSlices, f"{H}x{W}x{totalSlices}"))
        
slice_info_3d_df = pd.DataFrame(slice_info_3d, columns = ["BraTS21ID", "MGMT_Value","Modality","2D Shape","totalSlices", "FileShape"])

In [ ]:
slice_info_3d_df.totalSlices.value_counts()

In [ ]:
slice_info_3d_df.totalSlices.value_counts().sort_index()

In [ ]:
slice_info_3d_df.to_csv("slice_info_3d.csv",index = False)

In [ ]:
slice_info_3d_df

#### Check if patient information is present for all modalities

In [ ]:
tmp = trainPathDS.groupby("CaseId")["Modality"].agg(["nunique"]).reset_index()
tmp.columns = ["CaseId", "TotalMods"]
tmp[tmp.TotalMods < 4]

In [ ]:
tmp = testPathDS.groupby("CaseId")["Modality"].agg(["nunique"]).reset_index()
tmp.columns = ["CaseId", "TotalMods"]
tmp[tmp.TotalMods < 4]

In [ ]:
slice_info_3d_df[slice_info_3d_df.Modality == 'T1w']["totalSlices"].value_counts()

In [ ]:
import seaborn as sns
tmp = slice_info_3d_df["FileShape"].value_counts().reset_index().sort_values(by = "FileShape", ascending = False)
tmp= tmp.iloc[0:30]

#plt.figure(figsize = ())
sns.catplot(data = tmp, y = "index", x = "FileShape", kind="bar", aspect = 2, height = 15)

In [ ]:
slice_info_3d_df

In [ ]:
import seaborn as sns
tmp = slice_info_3d_df["2D Shape"].value_counts().reset_index().sort_values(by = "2D Shape", ascending = False)
tmp= tmp.iloc[0:30]

#plt.figure(figsize = ())
sns.catplot(data = tmp, y = "index", x = "2D Shape", kind="bar", aspect = 2, height = 15)

In [ ]:
import SimpleITK as sitk

In [ ]:
%%time

reader = sitk.ImageSeriesReader()
reader.LoadPrivateTagsOn()

for _type in types:
    filenamesDICOM = reader.GetGDCMSeriesFileNames(f'{rootFolder}/train/00000/{_type}')
    reader.SetFileNames(filenamesDICOM)
    t1_sitk = reader.Execute()
    sitk.WriteImage(t1_sitk,f'00000_{_type}.nii')

In [ ]:
from nilearn import plotting

for _type in types:
    plotting.plot_stat_map(f'{outputFolder}/00000_{_type}.nii', bg_img=None)

### Different orientation for same CaseId but different modality

In [ ]:
from nilearn.image import resample_to_img, reorder_img
import nibabel as nib

for _type in types:
    tmp = nib.load(f'{outputFolder}/00000_{_type}.nii')
    print(tmp.shape)
    tmp = reorder_img(tmp, resample="linear")
    print(tmp.shape)
    #plotting.plot_stat_map(tmp, bg_img=None)

### resample_to_img
- As the second modality is in RAS orientation, lets see if using second modality as base image Can I convert first image to second image
- And then check whether it is in RAS or not

In [ ]:
types

In [ ]:
from nilearn.image import resample_to_img, reorder_img
import nibabel as nib

base_image = nib.load(f'{outputFolder}/00000_T1w.nii')
image = nib.load(f'{outputFolder}/00000_FLAIR.nii')

new_img = resample_to_img(image, base_image, "linear")
print(new_img.shape)

new_img = reorder_img(new_img, resample="linear")
print(new_img.shape)

### Use dicom_nifty (but its not working at combine Slices
- Wanted to check if dicom images can be converted to nifti without saving the images

In [ ]:
'''import nibabel as nib
import dicom_numpy
import os
import numpy as np

pathtodicom = f'{rootFolder}/train/00000/FLAIR'
# get list of dicom images from directory that make up the 3D image
dicomlist = [pathtodicom + f for f in os.listdir(pathtodicom)]

# load dicom volume
vol, affine_LPS = dicom_numpy.combine_slices(dicomlist)

# convert the LPS affine to RAS
affine_RAS = np.diagflat([-1,-1,1,1]).dot(affine_LPS)

# create nibabel nifti object
niiimg = nib.Nifti1Image(vol, affine_RAS)
print(niiimg.shape)

niiimg = reorder_img(niiimg, resample="linear")
print(niiimg.shape)
#nib.save(niiimg, '/path/to/save')
'''

### Use dicom2nifti
- Convert directory method removes the flexibility of providing destination file name

In [ ]:
!pip install dicom2nifti

In [ ]:
import dicom2nifti

dicom_directory = f'{rootFolder}/train/00000/FLAIR'
output_folder = f'{outputFolder}'
dicom2nifti.convert_directory(dicom_directory, output_folder, compression=True, reorient=False)

'''
import nibabel as nib
nii = nib.load(output_folder)
print(nii.shape)

from nilearn.image import reorder_img

nii = reorder_img(nii, resample="linear")
print(nii.shape)
'''

### Manual sorting

In [ ]:
reader = sitk.ImageSeriesReader()
reader.LoadPrivateTagsOn()

_type = 'FLAIR'
allFiles = sorted(
                  getChildNames(f'{rootFolder}/train/00000/{_type}'), 
                  key=lambda x: int(x[:-4].split("-")[-1]),
              )

#filenamesDICOM = reader.GetGDCMSeriesFileNames(f'{rootFolder}/train/00000/{_type}')
filenamesDICOM = [f'{rootFolder}/train/00000/{_type}/{name}' for name in allFiles]
reader.SetFileNames(filenamesDICOM)
t1_sitk = reader.Execute()
sitk.WriteImage(t1_sitk,f'00000_{_type}_ms.nii')


from nilearn import plotting
plotting.plot_stat_map(f'{outputFolder}/00000_FLAIR_ms.nii', bg_img=None)

### Automatic sorting

In [ ]:
reader = sitk.ImageSeriesReader()
reader.LoadPrivateTagsOn()

_type = 'FLAIR'

filenamesDICOM = reader.GetGDCMSeriesFileNames(f'{rootFolder}/train/00000/{_type}')
reader.SetFileNames(filenamesDICOM)
t1_sitk = reader.Execute()
sitk.WriteImage(t1_sitk,f'00000_{_type}_as.nii')


from nilearn import plotting
plotting.plot_stat_map(f'{outputFolder}/00000_FLAIR_as.nii', bg_img=None)